<center>
    <h1> INF390 - Análisis Inteligente de Datos </h1>
    <h1> Tarea 1 - Visualización y Análisis Descriptivo de Datos con Python </h1>
    <h3>
        <table style="border: 0px;">
            <tr style="border: 0px;">
                <td style="text-align: center; border: 0px;"> Aldo Berrios Valenzuela </td>
                <td style="text-align: center; border: 0px; padding-right: 110px;"> </td>
                <td style="text-align: center; border: 0px;"> Lukas Zamora Jabre </td>
            </tr>
            <tr style="border: 0px;">
                <td style="text-align: center; border: 0px;"> 201304018-7 </td>
                <td style="text-align: center; border: 0px; padding-right: 110px;"> </td>
                <td style="text-align: center; border: 0px;"> 201430047-6 </td>
            </tr>
            <tr style="border: 0px;">
                <td style="text-align: center; font-size: 14px; border: 0px;"> `aldo.berrios.13@sansano.usm.cl` </td>
                <td style="text-align: center; border: 0px; padding-right: 110px;"> </td>
                <td style="text-align: center; font-size: 14px; border: 0px;"> `lukas.zamora.14@sansano.usm.cl` </td>
            </tr>
        </table>
    </h3> 
    <h4> Valparaíso, Viernes 29 de Octubre de 2017 </h4>
</center>

# 1. &nbsp;&nbsp;&nbsp;&nbsp; Preliminares

Para el desarrollo de esta tarea usaremos el dataset que se encuentra en https://archive.ics.uci.edu/ml/datasets/Census+Income. Luego, importamos las librerías que usaremos durante el desarrollo de la tarea:

In [50]:
import numpy as np
import pandas as pd
import scipy.stats as ss
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

A continuación, cargamos el dataset con pandas:

In [59]:
df = pd.DataFrame.from_csv('census_income_dataset.csv')
df = df.reset_index()

# 2. &nbsp;&nbsp;&nbsp;&nbsp; Desarrollo de Preguntas

## 2.1. &nbsp;&nbsp;&nbsp; Estudio de Estadística Descriptiva a 2 variables

El estudio de estadística descriptiva sólo podemos hacerlo para las variables cuantitativas del caso. Es por esto, que aquellas que sean del tipo categóricas las removemos de dataframe:

In [69]:
df_quantitative = df.select_dtypes(exclude=['object'])